In [1]:
import sqlite3, pandas as pd, numpy as np, nltk, string, matplotlib.pyplot as plt, seaborn as sns
import string, math, pickle
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from mlxtend.classifier import StackingClassifier
from pycm import *
from sklearn.model_selection import KFold
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
label = pd.read_csv("Activity_antibacterial.csv", index_col=0 )
data = pd.read_csv("Activity_training.csv", index_col=0 )

In [3]:
X = data
y = label.values.ravel()
bgc_ids = pd.Series(data.index)

In [4]:
rf = ensemble.RandomForestClassifier()
ets = ensemble.ExtraTreesClassifier()
nusvm = svm.NuSVC(probability=True)
ridge = linear_model.RidgeClassifierCV()
lr = linear_model.LogisticRegressionCV()

In [5]:
model = StackingClassifier(classifiers=[rf,ets,nusvm,ridge], meta_classifier= lr)

In [6]:
all_summary = []
class_summary = []
cv = KFold(n_splits=5,random_state=0,shuffle=True)
for i, (id_train, id_val) in enumerate(cv.split(bgc_ids)):
    train_ids, val_ids = bgc_ids[id_train], bgc_ids[id_val]
    X_train, X_val = data.loc[train_ids].values, data.loc[val_ids].values
    y_train, y_val = label.loc[train_ids].values, label.loc[val_ids].values
    print('Train:', len(X_train), 'Test:', len(X_val))
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    y_true = y_val.flatten()
    cm = ConfusionMatrix(actual_vector=y_true, predict_vector=y_pred)
    
    all_columns = pd.DataFrame(cm.overall_stat).columns
    all_ = pd.DataFrame(cm.overall_stat)[1:2].values
    all_summary.append(all_[0])
    
    class_columns = pd.DataFrame(cm.class_stat).columns
    class_ = pd.DataFrame(cm.class_stat)[1:2].values
    class_summary.append(class_[0])   

all_stat = pd.DataFrame(all_summary,columns=[all_columns]).mean()
all_stat = all_stat.T
class_stat = pd.DataFrame(class_summary,columns=[class_columns]).mean()
class_stat = class_stat.T

metric = pd.concat([all_stat, class_stat], axis=0)

metric.to_csv("metric_stacking_lr.csv")

Train: 321 Test: 81
Train: 321 Test: 81
Train: 322 Test: 80
Train: 322 Test: 80
Train: 322 Test: 80
